# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846853280281                   -0.70    4.5
  2   -7.852323168840       -2.26       -1.53    1.0
  3   -7.852646052977       -3.49       -2.52    3.2
  4   -7.852646677223       -6.20       -3.36    2.2
  5   -7.852646685803       -8.07       -4.77    1.0
  6   -7.852646686724       -9.04       -5.15    4.2
  7   -7.852646686729      -11.24       -5.95    2.0
  8   -7.852646686730      -12.45       -7.32    2.2
  9   -7.852646686730      -14.27       -7.51    2.2
 10   -7.852646686730   +    -Inf       -8.91    1.2
 11   -7.852646686730   +  -14.45       -9.53    3.2
 12   -7.852646686730      -14.75      -10.17    1.5
 13   -7.852646686730   +  -14.75      -10.94    1.5
 14   -7.852646686730   +  -14.57      -11.29    2.2
 15   -7.852646686730      -14.35      -12.17    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846706591151                   -0.70           5.0
  2   -7.852527264582       -2.24       -1.63   0.80    2.2
  3   -7.852637682714       -3.96       -2.71   0.80    1.0
  4   -7.852646469532       -5.06       -3.31   0.80    2.0
  5   -7.852646683459       -6.67       -4.23   0.80    2.0
  6   -7.852646686455       -8.52       -5.01   0.80    1.5
  7   -7.852646686727       -9.57       -5.90   0.80    2.2
  8   -7.852646686730      -11.62       -6.83   0.80    1.5
  9   -7.852646686730      -12.89       -7.51   0.80    2.2
 10   -7.852646686730   +    -Inf       -8.64   0.80    1.8
 11   -7.852646686730   +    -Inf       -8.68   0.80    2.8
 12   -7.852646686730      -15.05      -10.26   0.80    1.0
 13   -7.852646686730   +    -Inf      -11.16   0.80    2.5
 14   -7.852646686730   +  -15.05      -11.27   0.80    2.5
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.413991e+01     3.043367e+00
 * time: 0.5312039852142334
     1     8.518351e-01     1.693807e+00
 * time: 0.8184740543365479
     2    -1.680568e+00     2.064780e+00
 * time: 0.8579399585723877
     3    -3.806715e+00     1.597327e+00
 * time: 0.9126310348510742
     4    -5.039851e+00     1.500077e+00
 * time: 0.9690530300140381
     5    -6.687086e+00     9.284202e-01
 * time: 1.0228428840637207
     6    -7.406787e+00     4.171718e-01
 * time: 1.0843970775604248
     7    -7.685940e+00     2.642405e-01
 * time: 1.1274549961090088
     8    -7.785593e+00     1.056269e-01
 * time: 1.1663150787353516
     9    -7.826387e+00     1.099955e-01
 * time: 1.205129861831665
    10    -7.838431e+00     5.867780e-02
 * time: 1.2428560256958008
    11    -7.847187e+00     3.658418e-02
 * time: 1.282562017440796
    12    -7.850822e+00     3.562188e-02
 * time: 1.3209829330444336
    13    -7.851927e+00     2.473188e-02
 * time: 1.361512899398

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846825116848                   -0.70    4.5
  2   -7.852312763338       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686690                   -2.55
  2   -7.852646686730      -10.40       -5.98
  3   -7.852646686730   +    -Inf      -12.50


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.3625141478844024e-13
|ρ_newton - ρ_scfv| = 5.69928497821686e-13
|ρ_newton - ρ_dm|   = 3.717823759258426e-9
